## Dependencies

In [138]:
!pip install  lxml pandas
from urllib.request import urlopen
from lxml import html, etree
import pandas as pd
import numpy as np
import json

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Fund Portfolio
Configurable elements

In [139]:
funds=[]
funds.append({
        'Name':'Janus Henderson Global Technology Fund',
        'URL':'https://www.bloomberg.com/quote/JAGTX:US',
        'Allocation':4
    })
funds.append({
        'Name':'iShares Healthcare Innovation UCITS ETF',
        'URL':'https://www.bloomberg.com/quote/HEAL:SW',
        'Allocation':2
    })
funds.append({
        'Name':'Lyxor MSCI World Information Technology',
        'URL':'https://www.bloomberg.com/quote/TNOW:IM',
        'Allocation':2
    })
funds.append({
        'Name':'Lyxor MSCI World Consumer Discretionary',
        'URL':'https://www.bloomberg.com/quote/DISW:LN',
        'Allocation':4
    })


## Portfolio Analysis

In [140]:
def extractText(ele):
    return ele.text.strip()

In [141]:
def extractHoldingsFromStock(url):
    print(url)
    page=urlopen(url)
    tree = html.fromstring(page.read().decode('utf-8'))
    holding=tree.xpath('//div[@class="holding"]')[0]
    tbody=holding.xpath('//tbody')[0]
    holding={
        'FullName':tbody.xpath('//div[@data-type="full"]'),
        'Abbreviation':tbody.xpath('//div[@data-type="abbreviation"]'),
        'Percentage':tbody.xpath('//td[@data-type="percent"]')
    }
    df = pd.DataFrame(data=holding)
    df=df.apply(lambda s:s.apply(extractText))
    return df

In [142]:
df=pd.DataFrame(funds,columns=['Name','URL','Allocation'])
df

,Name,URL,Allocation
0,Janus Henderson Global Technology Fund,https://www.bloomberg.com/quote/JAGTX:US,4
1,iShares Healthcare Innovation UCITS ETF,https://www.bloomberg.com/quote/HEAL:SW,2
2,Lyxor MSCI World Information Technology,https://www.bloomberg.com/quote/TNOW:IM,2
3,Lyxor MSCI World Consumer Discretionary,https://www.bloomberg.com/quote/DISW:LN,4


In [143]:
df['Holdings']=df.URL.apply(extractHoldingsFromStock)

https://www.bloomberg.com/quote/JAGTX:US
https://www.bloomberg.com/quote/HEAL:SW
https://www.bloomberg.com/quote/TNOW:IM
https://www.bloomberg.com/quote/DISW:LN


In [144]:
sum=df.Allocation.sum()
weights=list(map(lambda v:round(v/sum,2),df.Allocation))
df['OverallWeight']=weights
df[['Name','OverallWeight']]

,Name,OverallWeight
0,Janus Henderson Global Technology Fund,0.33
1,iShares Healthcare Innovation UCITS ETF,0.17
2,Lyxor MSCI World Information Technology,0.17
3,Lyxor MSCI World Consumer Discretionary,0.33


In [145]:
df

,Name,URL,Allocation,Holdings,OverallWeight
0,Janus Henderson Global Technology Fund,https://www.bloomberg.com/quote/JAGTX:US,4,FullName Abbreviation...,0.33
1,iShares Healthcare Innovation UCITS ETF,https://www.bloomberg.com/quote/HEAL:SW,2,FullName Abbreviation...,0.17
2,Lyxor MSCI World Information Technology,https://www.bloomberg.com/quote/TNOW:IM,2,FullName Abbreviation Percentage 0...,0.17
3,Lyxor MSCI World Consumer Discretionary,https://www.bloomberg.com/quote/DISW:LN,4,FullName Abbreviation...,0.33


In [146]:
def factorInWeights(row):
#     print(row.Name)
    row.Holdings['OverallPercentage']=row.Holdings.Percentage.apply(lambda v:float(v)*row.OverallWeight)
    return 

df.apply(factorInWeights,axis=1)

0    None
1    None
2    None
3    None
dtype: object

In [147]:
def appendFundName(row):
    row.Holdings['FundName']=row.Name
    return
df.apply(appendFundName,axis=1)

0    None
1    None
2    None
3    None
dtype: object

In [148]:
allHoldings=pd.concat(df.Holdings.values)
# allHoldings

In [149]:
grouped=allHoldings.groupby(
    ['Abbreviation']
).agg({
    'FullName':np.unique,
    'OverallPercentage': [np.sum,np.count_nonzero],
    'FundName':np.unique
}).rename(
    columns={
        'OverallPercentage':'Overall',
        'count_nonzero':'count',
        'unique':'values'
    }).reset_index()
# grouped

In [150]:
final=grouped.sort_values([('Overall', 'sum'),('Overall', 'count')],ascending=[0,0])
final.columns
final[['Abbreviation','FullName', 'FundName', 'Overall']]


Abbreviation                        FullName  \
                                         values   
12      AMZN:US                  Amazon.com Inc   
30      MSFT:US                  Microsoft Corp   
22      GOOG:US                    Alphabet Inc   
9       AAPL:US                       Apple Inc   
24        HD:US              Home Depot Inc/The   
7        700:HK            Tencent Holdings Ltd   
17       CRM:US              salesforce.com Inc   
8       7203:JP               Toyota Motor Corp   
14      BABA:US       Alibaba Group Holding Ltd   
21        FB:US                    Facebook Inc   
11      ADBE:US               Adobe Systems Inc   
19       DIS:US              Walt Disney Co/The   
29      MCHP:US        Microchip Technology Inc   
32      NFLX:US                     Netflix Inc   
16     CMCSA:US                    Comcast Corp   
4       2330:TT  Taiwan Semiconductor Manufactu   
13       APH:US                   Amphenol Corp   
28       MCD:US                 McDonald's Corp   
23     GOOGL:US                    Alphabet Inc   
15      BKNG:US            Booking Holdings Inc   
27        MC:FP  LVMH Moet Hennessy Louis Vuitt   
33       NKE:US                        NIKE Inc   
25      INTC:US                      Intel Corp   
36         V:US                        Visa Inc   
20      EUR:CUR                            EURO   
18      CSCO:US               Cisco Systems Inc   
0     068270:KS                   Celltrion Inc   
10      ABMD:US                     ABIOMED Inc   
1     084990:KS                  ViroMed Co Ltd   
35       SRX:AU              Sirtex Medical Ltd   
26        MA:US                  Mastercard Inc   
34      NVDA:US                     NVIDIA Corp   
2       1093:HK   CSPC Pharmaceutical Group Ltd   
3     207940:KS        Samsung Biologics Co Ltd   
6       4974:JP                  Takara Bio Inc   
5       3933:HK  United Laboratories Internatio   
31      MYGN:US             Myriad Genetics Inc   

                                             FundName Overall        
                                               values     sum count  
12  [Janus Henderson Global Technology Fund, Lyxor...  5.1546   2.0  
30  [Janus Henderson Global Technology Fund, Lyxor...  3.4806   2.0  
22  [Janus Henderson Global Technology Fund, Lyxor...  2.5672   2.0  
9             Lyxor MSCI World Information Technology  2.2151   1.0  
24            Lyxor MSCI World Consumer Discretionary  1.3926   1.0  
7              Janus Henderson Global Technology Fund  1.3827   1.0  
17             Janus Henderson Global Technology Fund  1.2606   1.0  
8             Lyxor MSCI World Consumer Discretionary  1.0989   1.0  
14             Janus Henderson Global Technology Fund  1.0692   1.0  
21            Lyxor MSCI World Information Technology  1.0166   1.0  
11             Janus Henderson Global Technology Fund  0.9867   1.0  
19            Lyxor MSCI World Consumer Discretionary  0.9834   1.0  
29             Janus Henderson Global Technology Fund  0.9669   1.0  
32            Lyxor MSCI World Consumer Discretionary  0.9636   1.0  
16            Lyxor MSCI World Consumer Discretionary  0.9405   1.0  
4              Janus Henderson Global Technology Fund  0.8877   1.0  
13             Janus Henderson Global Technology Fund  0.8646   1.0  
28            Lyxor MSCI World Consumer Discretionary  0.8250   1.0  
23            Lyxor MSCI World Information Technology  0.7429   1.0  
15            Lyxor MSCI World Consumer Discretionary  0.6468   1.0  
27            Lyxor MSCI World Consumer Discretionary  0.6336   1.0  
33            Lyxor MSCI World Consumer Discretionary  0.6039   1.0  
25            Lyxor MSCI World Information Technology  0.5882   1.0  
36            Lyxor MSCI World Information Technology  0.5491   1.0  
20            iShares Healthcare Innovation UCITS ETF  0.5032   1.0  
18            Lyxor MSCI World Information Technology  0.4947   1.0  
0             iShares Healthcare Innovation UCITS ETF

## Git Commit

In [153]:
!git add *.ipynb
!git commit -am "Final"
!git push

[binder 9ca0982] Final
 1 file changed, 980 insertions(+)
 create mode 100644 notebook/ExampleProject.ipynb
Counting objects: 4, done.
Delta compression using up to 32 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 5.01 KiB | 5.01 MiB/s, done.
Total 4 (delta 0), reused 0 (delta 0)
To https://github.com/yoyu777/jupyter_notebooks
   adb05b1..9ca0982  binder -> binder
